In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lazypredict
from lazypredict.Supervised import LazyClassifier
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

/home/arma/anaconda3/envs/teachopencadd/lib/python3.9/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/arma/anaconda3/envs/teachopencadd/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
# Set path to this notebook
HERE = Path(_dh[-1])
levels_up = 3
HDAC6= HERE.parents[levels_up-1]/'input'
HDAC6

PosixPath('/home/arma/hDAC/HDAC/Activity/HDAC6/input')

In [5]:
df1 = pd.read_pickle(HDAC6/"HDAC6_1024B.csv")
df1 = df1.sample(frac=1).reset_index(drop=True)
df1.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,molecular_weight,n_rot,n_heavy,n_hba,n_hbd,logp,num_ar,num_sa,num_alip,pchembl
0,CHEMBL504040,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[100053, 10511, 1733887, 904525, 692898, 28201...",876.56,14.00,62.00,16.00,7.00,3.26,2.00,2.00,2.00,6.93
1,CHEMBL4282673,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 47126, 2064333, 12474601, 6528194, ...",342.05,7.00,20.00,4.00,2.00,1.51,2.00,0.00,0.00,7.72
2,CHEMBL4070745,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[5196932, 7212973, 3844537, 7125875, 28861, 15...",362.16,9.00,26.00,6.00,4.00,2.36,2.00,0.00,0.00,8.21
3,CHEMBL466459,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[1122035, 1085935, 836109, 12913187, 19372760,...",562.18,8.00,41.00,10.00,5.00,3.83,4.00,0.00,0.00,5.77
4,CHEMBL3110005,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 28470141, 37247964, 73736535, 36956...",253.11,3.00,19.00,2.00,2.00,2.69,2.00,1.00,1.00,5.01


In [6]:
df1[df1['molecule_chembl_id'] == 'CHEMBL2105763']

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,molecular_weight,n_rot,n_heavy,n_hba,n_hbd,logp,num_ar,num_sa,num_alip,pchembl
2030,CHEMBL2105763,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4277439, 2478511, 1683617, 1179055, 492964, 2...",394.21,6.00,29.00,7.00,3.00,2.09,3.00,1.00,1.00,6.46


In [7]:
# Add column for activity
df1["activity"] = np.zeros(len(df1))

# Mark every molecule as active with an pIC50 of >= 6.6, 0 otherwise
df1.loc[df1[df1.pchembl >= 6.6].index, "activity"] = 1.0

# NBVAL_CHECK_OUTPUT
print("Number of active compounds:", int(df1.activity.sum()))
print("Number of inactive compounds:", len(df1) - int(df1.activity.sum()))

Number of active compounds: 1837
Number of inactive compounds: 1134


In [8]:
df1.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,molecular_weight,n_rot,n_heavy,n_hba,n_hbd,logp,num_ar,num_sa,num_alip,pchembl,activity
0,CHEMBL504040,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[100053, 10511, 1733887, 904525, 692898, 28201...",876.56,14.00,62.00,16.00,7.00,3.26,2.00,2.00,2.00,6.93,1.00
1,CHEMBL4282673,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 47126, 2064333, 12474601, 6528194, ...",342.05,7.00,20.00,4.00,2.00,1.51,2.00,0.00,0.00,7.72,1.00
2,CHEMBL4070745,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[5196932, 7212973, 3844537, 7125875, 28861, 15...",362.16,9.00,26.00,6.00,4.00,2.36,2.00,0.00,0.00,8.21,1.00
3,CHEMBL466459,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[1122035, 1085935, 836109, 12913187, 19372760,...",562.18,8.00,41.00,10.00,5.00,3.83,4.00,0.00,0.00,5.77,0.00
4,CHEMBL3110005,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 28470141, 37247964, 73736535, 36956...",253.11,3.00,19.00,2.00,2.00,2.69,2.00,1.00,1.00,5.01,0.00


#### By using MACCS fingerprints that are 166 bits 

In [9]:
X1 = np.array(list((df1['fp_MACCS']))).astype(float)
#X.shape
Y1 = df1["activity"].values
#y.shape

In [10]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, Y1, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train1.shape)
print("Shape of test data:", X_test1.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 167)
Shape of test data: (595, 167)


In [11]:
# Defines and builds the LazyClassifier
clf1 =  LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
model1,predictions1 = clf1.fit(X_train1, X_test1, Y_train1, Y_test1)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:07<00:01,  2.75it/s]

[19:03:57] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [02:17<00:00,  4.74s/it]


In [12]:
predictions1.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.83,0.82,0.82,0.83,57.03
RandomForestClassifier,0.82,0.81,0.81,0.82,0.27
BaggingClassifier,0.82,0.81,0.81,0.82,0.18
XGBClassifier,0.82,0.81,0.81,0.82,72.59
ExtraTreesClassifier,0.81,0.81,0.81,0.81,0.34
KNeighborsClassifier,0.81,0.80,0.80,0.81,0.28
NuSVC,0.81,0.79,0.79,0.81,0.69
SVC,0.81,0.79,0.79,0.81,0.72
DecisionTreeClassifier,0.79,0.78,0.78,0.79,0.04


In [13]:
predictions1.to_csv('LazyPredict_MACCS.csv')

### USING 1024 BITS FINGERPRINTS

In [14]:
#By using Morgan fingerprints with radius of 3 and 1024 bits
X2 = np.array(list((df1['fp_Morgan3']))).astype(float)
#X.shape
Y2 = df1["activity"].values
#y.shape

In [15]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y2, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train2.shape)
print("Shape of test data:", X_test2.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 1024)
Shape of test data: (595, 1024)


In [16]:
clf2 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models2, predictions2 = clf2.fit(X_train2, X_test2, Y_train2, Y_test2)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:27<00:05,  1.75s/it]

[19:06:34] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [03:25<00:00,  7.08s/it]


In [17]:
predictions2.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.85,0.83,0.83,0.84,0.60
ExtraTreesClassifier,0.84,0.82,0.82,0.84,0.75
LGBMClassifier,0.83,0.82,0.82,0.83,148.83
XGBClassifier,0.82,0.80,0.80,0.82,28.81
BaggingClassifier,0.82,0.81,0.81,0.82,0.88
SVC,0.80,0.78,0.78,0.80,3.87
NuSVC,0.80,0.78,0.78,0.80,4.33
DecisionTreeClassifier,0.77,0.76,0.76,0.78,0.27
RidgeClassifierCV,0.77,0.76,0.76,0.77,1.81


In [18]:
predictions2.to_csv('LazyPredict_Morgan3.csv')

In [19]:
#By using Morgan fingerprints with radius of 3 and 1024 bits and feautures
X3 = np.array(list((df1['fp_MorganF']))).astype(float)
#X.shape
Y3 = df1["activity"].values
#y.shape

In [20]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X3, Y3, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train3.shape)
print("Shape of test data:", X_test3.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 1024)
Shape of test data: (595, 1024)


In [21]:
clf3 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models3, predictions3 = clf3.fit(X_train3, X_test3, Y_train3, Y_test3)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:39<00:06,  2.17s/it]

[19:10:11] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [04:32<00:00,  9.41s/it]


In [22]:
predictions3.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.84,0.83,0.83,0.84,47.65
RandomForestClassifier,0.84,0.83,0.83,0.84,0.77
LGBMClassifier,0.83,0.83,0.83,0.83,185.92
ExtraTreesClassifier,0.83,0.82,0.82,0.83,1.07
BaggingClassifier,0.83,0.83,0.83,0.83,1.02
DecisionTreeClassifier,0.81,0.81,0.81,0.81,0.34
NuSVC,0.80,0.78,0.78,0.80,4.72
SVC,0.80,0.78,0.78,0.80,4.05
RidgeClassifierCV,0.77,0.77,0.77,0.77,3.83


In [23]:
predictions3.to_csv('LazyPredict_MorganF.csv')

In [24]:
#By using MAP4 fingerprints with 1024 bits
X4 = np.array(list((df1['fp_MAP4']))).astype(float)
#X.shape
Y4 = df1["activity"].values
#y.shape

In [25]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X4, Y4, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train4.shape)
print("Shape of test data:", X_test4.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 1024)
Shape of test data: (595, 1024)


In [26]:
# Defines and builds the lazyclassifier
clf4 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models4, predictions4 = clf4.fit(X_train4, X_test4, Y_train4, Y_test4)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:07<00:09,  3.27s/it]

[19:15:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [05:01<00:00, 10.41s/it]


In [27]:
predictions4.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.82,0.81,0.81,0.82,178.60
ExtraTreesClassifier,0.82,0.80,0.80,0.82,0.92
XGBClassifier,0.81,0.80,0.80,0.81,55.48
RandomForestClassifier,0.81,0.79,0.79,0.81,2.94
BaggingClassifier,0.79,0.78,0.78,0.79,10.69
KNeighborsClassifier,0.78,0.75,0.75,0.77,1.86
AdaBoostClassifier,0.76,0.75,0.75,0.77,6.60
NuSVC,0.76,0.74,0.74,0.76,4.72
DecisionTreeClassifier,0.75,0.73,0.73,0.75,1.78


In [28]:
predictions4.to_csv('LazyPredict_MAP4.csv')

## USE 2048 BITS FINGERPRINTS

In [29]:
df2 = pd.read_pickle(HDAC6/"HDAC6_2048B.csv")
df2 = df2.sample(frac=1).reset_index(drop=True)
df2

,molecule_chembl_id,fp_Morgan3,fp_MorganF,fp_MAP4,molecular_weight,n_rot,n_heavy,n_hba,n_hbd,logp,num_ar,num_sa,num_alip,pchembl
0,CHEMBL99,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 3429503, 7998790, 6649368, 8735147,...",302.16,6.00,22.00,4.00,2.00,2.58,1.00,0.00,0.00,9.00
1,CHEMBL4286053,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10514595, 47126, 2064333, 12474601, 24666102,...",314.02,5.00,18.00,4.00,2.00,0.73,2.00,0.00,0.00,6.45
2,CHEMBL1097369,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13333824, 10427556, 6095547, 5213123, 2452922...",361.18,4.00,27.00,4.00,2.00,4.73,3.00,1.00,1.00,7.60
3,CHEMBL3908767,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2032543, 26371649, 3707928, 4020886, 501966, ...",397.18,6.00,29.00,7.00,3.00,1.89,3.00,1.00,1.00,5.27
4,CHEMBL4174193,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[5976924, 3429503, 4246646, 453610, 9204552, 2...",341.17,6.00,25.00,3.00,3.00,3.50,2.00,0.00,0.00,7.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2966,CHEMBL4162282,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[4813367, 3821889, 8033062, 6078715, 583588, 2...",460.21,8.00,34.00,5.00,3.00,3.46,3.00,0.00,0.00,8.52
2967,CHEMBL140525,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[5547132, 1675066, 3082320, 3724449, 7332227, ...",485.21,11.00,36.00,6.00,3.00,4.38,4.00,0.00,0.00,6.11
2968,CHEMBL3891420,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...","[4681989, 2349897, 3921740, 116276, 4556712, 2...",470.21,6.00,35.00,6.00,4.00,3.25,4.00,1.00,1.00,8.55
2969,CHEMBL4447317,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[8980709, 7477158, 5660089, 7541421, 1206207, ...",701.12,10.00,48.00,5.00,2.00,6.55,4.00,0.00,0.00,6.82


In [30]:
# Add column for activity
df2["activity"] = np.zeros(len(df2))

# Mark every molecule as active with an pIC50 of >= 6.6, 0 otherwise
df2.loc[df2[df2.pchembl >= 6.6].index, "activity"] = 1.0

# NBVAL_CHECK_OUTPUT
print("Number of active compounds:", int(df2.activity.sum()))
print("Number of inactive compounds:", len(df2) - int(df2.activity.sum()))

Number of active compounds: 1837
Number of inactive compounds: 1134


In [31]:
#2048 bits Morgan fingerprints with radius of 3
X5 = np.array(list((df2['fp_Morgan3']))).astype(float)
#X.shape
Y5 = df2["activity"].values
#y.shape

In [32]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X5, Y5, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train5.shape)
print("Shape of test data:", X_test5.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 2048)
Shape of test data: (595, 2048)


In [33]:
clf5 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models5, predictions5 = clf5.fit(X_train5, X_test5, Y_train5, Y_test5)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:35<00:17,  5.86s/it]

[19:20:43] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [05:28<00:00, 11.34s/it]


In [34]:
predictions5.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.87,0.86,0.86,0.87,1.13
ExtraTreesClassifier,0.86,0.85,0.85,0.86,1.71
LGBMClassifier,0.85,0.84,0.84,0.85,188.07
XGBClassifier,0.84,0.83,0.83,0.84,45.16
NuSVC,0.84,0.82,0.82,0.84,10.12
SVC,0.84,0.81,0.81,0.83,9.57
BaggingClassifier,0.82,0.81,0.81,0.82,2.93
BernoulliNB,0.81,0.81,0.81,0.82,0.15
CalibratedClassifierCV,0.82,0.78,0.78,0.81,15.76


In [35]:
predictions5.to_csv('LazyPredict_Morgan3_2048.csv')

In [36]:
#2048 bits Morgan fingerprints with feautures
X6 = np.array(list((df2['fp_MorganF']))).astype(float)
#X.shape
Y6 = df2["activity"].values
#y.shape

In [37]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train6, X_test6, Y_train6, Y_test6 = train_test_split(X6, Y6, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train6.shape)
print("Shape of test data:", X_test6.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 2048)
Shape of test data: (595, 2048)


In [38]:
clf6 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models6, predictions6 = clf6.fit(X_train6, X_test6, Y_train6, Y_test6)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:23<00:16,  5.59s/it]

[19:26:00] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [04:43<00:00,  9.78s/it]


In [39]:
predictions6.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.87,0.85,0.85,0.86,1.09
LGBMClassifier,0.86,0.85,0.85,0.86,158.22
ExtraTreesClassifier,0.86,0.85,0.85,0.86,1.46
XGBClassifier,0.85,0.84,0.84,0.85,41.53
BaggingClassifier,0.83,0.82,0.82,0.83,2.08
SVC,0.83,0.80,0.80,0.83,8.83
AdaBoostClassifier,0.83,0.81,0.81,0.83,2.50
NuSVC,0.83,0.80,0.80,0.82,10.08
NearestCentroid,0.81,0.81,0.81,0.81,0.16


In [40]:
predictions6.to_csv('LazyPredict_MorganF_2048.csv')

In [41]:
#2048 bits MAP4 fingerprints
X7 = np.array(list((df2['fp_MAP4']))).astype(float)
#X.shape
Y7 = df2["activity"].values
#y.shape

In [42]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train7, X_test7, Y_train7, Y_test7 = train_test_split(X7, Y7, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train7.shape)
print("Shape of test data:", X_test7.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 2048)
Shape of test data: (595, 2048)


In [43]:
# Defines and builds the lazyclassifier
clf7 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models7, predictions7 = clf7.fit(X_train7, X_test7, Y_train7, Y_test7)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [02:08<00:16,  5.42s/it]

[19:31:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [06:49<00:00, 14.13s/it]


In [44]:
predictions7.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.85,0.83,0.83,0.85,4.47
LGBMClassifier,0.84,0.83,0.83,0.84,230.08
XGBClassifier,0.84,0.83,0.83,0.84,51.35
ExtraTreesClassifier,0.83,0.81,0.81,0.83,1.57
BaggingClassifier,0.82,0.81,0.81,0.82,24.43
NuSVC,0.82,0.80,0.80,0.82,10.13
KNeighborsClassifier,0.81,0.79,0.79,0.81,3.65
SVC,0.81,0.79,0.79,0.81,8.82
AdaBoostClassifier,0.81,0.80,0.80,0.81,13.77


In [45]:
predictions7.to_csv('LazyPredict_MAP4_2048.csv')

### By combining 1D descriptors - 9 of them -  with fingerprints of 1024 bits

In [46]:
fp = np.array(list((df1['fp_MACCS']))).astype(float)
oneD = np.array((df1[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X8 = np.column_stack((fp,oneD))
Y8 = df1["activity"].values
X8.shape

(2971, 176)

In [47]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train8, X_test8, Y_train8, Y_test8 = train_test_split(X8, Y8, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train8.shape)
print("Shape of test data:", X_test8.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 176)
Shape of test data: (595, 176)


In [48]:
# Defines and builds the lazyclassifier
clf8 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models8, predictions8 = clf8.fit(X_train8, X_test8, Y_train8, Y_test8)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:09<00:01,  2.19it/s]

[19:36:18] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [04:00<00:00,  8.30s/it]


In [49]:
predictions8.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.83,0.83,0.83,0.83,0.27
LGBMClassifier,0.83,0.81,0.81,0.83,175.11
RandomForestClassifier,0.83,0.81,0.81,0.82,0.30
KNeighborsClassifier,0.82,0.81,0.81,0.82,0.28
XGBClassifier,0.82,0.81,0.81,0.82,56.62
SVC,0.81,0.79,0.79,0.81,0.85
NuSVC,0.81,0.79,0.79,0.81,0.72
BaggingClassifier,0.79,0.79,0.79,0.79,0.22
ExtraTreeClassifier,0.78,0.79,0.79,0.79,0.02


In [50]:
predictions8.to_csv('LazyPredict_MACCS_1Dadded.csv')

In [51]:
#with Morgan 1024 bits fingerprints
fp = np.array(list((df1['fp_Morgan3']))).astype(float)
oneD = np.array((df1[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X9 = np.column_stack((fp,oneD))
Y9 = df1["activity"].values
X9.shape

(2971, 1033)

In [52]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train9, X_test9, Y_train9, Y_test9 = train_test_split(X9, Y9, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train9.shape)
print("Shape of test data:", X_test9.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 1033)
Shape of test data: (595, 1033)


In [53]:
clf9 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models9, predictions9 = clf9.fit(X_train9, X_test9, Y_train9, Y_test9)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:40<00:08,  2.70s/it]

[19:40:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [04:39<00:00,  9.64s/it]


In [54]:
predictions9.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.84,0.82,0.82,0.84,0.54
ExtraTreesClassifier,0.83,0.81,0.81,0.83,0.75
BaggingClassifier,0.82,0.82,0.82,0.82,0.83
LGBMClassifier,0.82,0.81,0.81,0.82,187.97
XGBClassifier,0.82,0.80,0.80,0.82,50.82
SVC,0.81,0.79,0.79,0.80,4.13
NuSVC,0.80,0.78,0.78,0.80,4.69
DecisionTreeClassifier,0.79,0.78,0.78,0.79,0.19
RidgeClassifierCV,0.77,0.76,0.76,0.77,5.95


In [55]:
predictions9.to_csv('LazyPredict_Morgan3_1Dadded.csv')

In [56]:
#with Morgan 1024 bits fingerprints with feautures
fp = np.array(list((df1['fp_MorganF']))).astype(float)
oneD = np.array((df1[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X10 = np.column_stack((fp,oneD))
Y10 = df1["activity"].values
X10.shape

(2971, 1033)

In [57]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train10, X_test10, Y_train10, Y_test10 = train_test_split(X10, Y10, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train10.shape)
print("Shape of test data:", X_test10.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 1033)
Shape of test data: (595, 1033)


In [58]:
clf10 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models10, predictions10 = clf10.fit(X_train10, X_test10, Y_train10, Y_test10)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [00:44<00:08,  2.73s/it]

[19:45:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [04:39<00:00,  9.63s/it]


In [59]:
predictions10.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.84,0.83,0.83,0.84,0.54
LGBMClassifier,0.84,0.83,0.83,0.84,184.90
ExtraTreesClassifier,0.84,0.82,0.82,0.84,0.56
XGBClassifier,0.83,0.83,0.83,0.83,49.59
BaggingClassifier,0.82,0.82,0.82,0.83,0.86
DecisionTreeClassifier,0.81,0.81,0.81,0.81,0.18
NuSVC,0.80,0.78,0.78,0.80,4.71
SVC,0.80,0.78,0.78,0.80,4.06
RidgeClassifierCV,0.77,0.77,0.77,0.78,5.98


In [60]:
predictions10.to_csv('LazyPredict_MorganF_1Dadded.csv')

In [61]:
fp = np.array(list((df1['fp_MAP4']))).astype(float)
oneD = np.array((df1[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X11 = np.column_stack((fp,oneD))
Y11 = df1["activity"].values
X11.shape

(2971, 1033)

In [62]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train11, X_test11, Y_train11, Y_test11 = train_test_split(X11, Y11, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train11.shape)
print("Shape of test data:", X_test11.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 1033)
Shape of test data: (595, 1033)


In [63]:
clf11 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models11, predictions11 = clf11.fit(X_train11, X_test11, Y_train11, Y_test11)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:00<00:07,  2.37s/it]

[19:50:30] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [04:57<00:00, 10.25s/it]


In [64]:
predictions11.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.83,0.81,0.81,0.83,181.22
ExtraTreesClassifier,0.82,0.79,0.79,0.81,1.02
RandomForestClassifier,0.82,0.79,0.79,0.81,2.96
XGBClassifier,0.81,0.80,0.80,0.81,55.05
BaggingClassifier,0.78,0.77,0.77,0.78,10.61
KNeighborsClassifier,0.78,0.76,0.76,0.78,2.31
AdaBoostClassifier,0.76,0.75,0.75,0.77,6.06
NuSVC,0.77,0.74,0.74,0.77,4.63
SVC,0.76,0.73,0.73,0.76,4.28


In [65]:
predictions11.to_csv('LazyPredict_MAP4_1Dadded.csv')

### By combining 1D descriptors - 9 of them -  with fingerprints of 2048 bits

In [66]:
#with Morgan 2048 bits fingerprints
fp = np.array(list((df2['fp_Morgan3']))).astype(float)
oneD = np.array((df2[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X12 = np.column_stack((fp,oneD))
Y12 = df2["activity"].values
X12.shape

(2971, 2057)

In [67]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train12, X_test12, Y_train12, Y_test12 = train_test_split(X12, Y12, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train12.shape)
print("Shape of test data:", X_test12.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 2057)
Shape of test data: (595, 2057)


In [68]:
clf12 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models12, predictions12 = clf12.fit(X_train12, X_test12, Y_train12, Y_test12)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:34<00:18,  6.24s/it]

[19:56:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [03:43<00:00,  7.69s/it]


In [69]:
predictions12.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.87,0.86,0.86,0.87,1.02
RandomForestClassifier,0.87,0.85,0.85,0.86,0.73
XGBClassifier,0.85,0.84,0.84,0.85,45.84
LGBMClassifier,0.85,0.84,0.84,0.85,82.29
BaggingClassifier,0.85,0.84,0.84,0.85,1.74
NuSVC,0.84,0.82,0.82,0.84,10.17
SVC,0.83,0.81,0.81,0.83,9.75
CalibratedClassifierCV,0.82,0.79,0.79,0.81,15.07
NearestCentroid,0.81,0.81,0.81,0.81,0.16


In [70]:
predictions12.to_csv('LazyPredict_Morgan3_2048_1Dadded.csv')

In [71]:
#with Morgan 2048 bits fingerprints with feautures
fp = np.array(list((df2['fp_MorganF']))).astype(float)
oneD = np.array((df2[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X13 = np.column_stack((fp,oneD))
Y13 = df2["activity"].values
X13.shape

(2971, 2057)

In [72]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train13, X_test13, Y_train13, Y_test13 = train_test_split(X13, Y13, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train13.shape)
print("Shape of test data:", X_test13.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 2057)
Shape of test data: (595, 2057)


In [73]:
clf13 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models13, predictions13 = clf13.fit(X_train13, X_test13, Y_train13, Y_test13)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [01:39<00:20,  6.71s/it]

[19:59:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [05:03<00:00, 10.48s/it]


In [74]:
predictions13.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.87,0.85,0.85,0.87,0.73
ExtraTreesClassifier,0.86,0.84,0.84,0.86,0.93
LGBMClassifier,0.86,0.84,0.84,0.86,158.63
XGBClassifier,0.85,0.84,0.84,0.85,45.61
BaggingClassifier,0.83,0.82,0.82,0.83,1.49
SVC,0.83,0.81,0.81,0.83,8.69
NuSVC,0.83,0.80,0.80,0.83,10.42
AdaBoostClassifier,0.83,0.81,0.81,0.83,1.39
NearestCentroid,0.81,0.81,0.81,0.81,0.18


In [75]:
predictions13.to_csv('LazyPredict_MorganF_2048_1Dadded.csv')

In [76]:
#with MAP4 2048 bits fingerprints
fp = np.array(list((df2['fp_MAP4']))).astype(float)
oneD = np.array((df2[['molecular_weight', 'n_rot','n_heavy', 'n_hba', 'n_hbd', 'logp', 'num_ar', 'num_sa', 'num_alip' ]])).astype(float)
X14 = np.column_stack((fp,oneD))
Y14 = df2["activity"].values
X14.shape

(2971, 2057)

In [77]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train14, X_test14, Y_train14, Y_test14 = train_test_split(X14, Y14, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train14.shape)
print("Shape of test data:", X_test14.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (2376, 2057)
Shape of test data: (595, 2057)


In [78]:
clf14 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models14, predictions14 = clf14.fit(X_train14, X_test14, Y_train14, Y_test14)

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                  | 26/29 [02:11<00:17,  5.83s/it]

[20:05:25] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [06:57<00:00, 14.40s/it]


In [79]:
predictions14.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.85,0.83,0.83,0.85,4.34
LGBMClassifier,0.85,0.83,0.83,0.85,229.84
XGBClassifier,0.84,0.82,0.82,0.84,56.57
ExtraTreesClassifier,0.83,0.81,0.81,0.83,1.12
NuSVC,0.83,0.81,0.81,0.83,10.11
KNeighborsClassifier,0.82,0.79,0.79,0.81,4.57
BaggingClassifier,0.81,0.80,0.80,0.81,22.56
SVC,0.81,0.79,0.79,0.81,8.95
AdaBoostClassifier,0.81,0.80,0.80,0.81,12.52


In [80]:
predictions14.to_csv('LazyPredict_MAP4_2048_1Dadded.csv')